# MRI Preprocessing Scripts 8-10
## Complete Working Version

In [9]:
!pip install -q nibabel nilearn SimpleITK scikit-learn pandas

In [10]:
import numpy as np
import nibabel as nib
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from nilearn import datasets
from scipy import ndimage
from skimage import morphology
import SimpleITK as sitk
from sklearn.mixture import GaussianMixture
from concurrent.futures import ProcessPoolExecutor
from datetime import datetime
from nibabel.orientations import io_orientation, axcodes2ornt, ornt_transform
print("Imports complete")

Imports complete


In [11]:
# Setup
OUTPUT_DIR = Path('/Users/aazeez/Documents/Personal/ABDN_2025/content5/derivatives')
OUTPUT_DIR.mkdir(exist_ok=True)
print(f"Output: {OUTPUT_DIR.absolute()}")

Output: /Users/aazeez/Documents/Personal/ABDN_2025/content5/derivatives


# Helper Functions

In [12]:
def reorient_img(img):
    curr = io_orientation(img.affine)
    targ = axcodes2ornt('RAS')
    trans = ornt_transform(curr, targ)
    data = nib.apply_orientation(img.get_fdata(), trans)
    aff = img.affine @ nib.orientations.inv_ornt_aff(trans, img.shape)
    return nib.Nifti1Image(data, aff)

def correct_bias(img):
    s = sitk.GetImageFromArray(img.get_fdata().T)
    s = sitk.Cast(s, sitk.sitkFloat32)
    c = sitk.N4BiasFieldCorrectionImageFilter()
    c.SetMaximumNumberOfIterations([50,50,50,50])
    r = c.Execute(s)
    return nib.Nifti1Image(sitk.GetArrayFromImage(r).T, img.affine)

def strip_skull(img):
    d = img.get_fdata()
    t = np.percentile(d[d>0], 20)
    m = d > t
    m = morphology.remove_small_objects(m, 1000)
    m = ndimage.binary_fill_holes(m)
    l, _ = ndimage.label(m)
    s = ndimage.sum(m, l, range(l.max()+1))
    m = l == np.argmax(s)
    m = ndimage.binary_closing(m, iterations=3)
    return nib.Nifti1Image(d*m, img.affine), nib.Nifti1Image(m.astype(np.uint8), img.affine)

def segment_img(img, mask):
    d = img.get_fdata()
    md = mask.get_fdata().astype(bool)
    v = d[md].reshape(-1, 1)
    vn = (v - v.mean()) / v.std()
    g = GaussianMixture(n_components=3, random_state=42)
    g.fit(vn)
    l = g.predict(vn)
    s = np.zeros(d.shape, dtype=np.uint8)
    s[md] = l + 1
    si = np.argsort(g.means_.flatten())
    ss = np.zeros_like(s)
    for i, idx in enumerate(si):
        ss[s == idx+1] = i+1
    return nib.Nifti1Image(ss, img.affine)

print("Helper functions defined")

Helper functions defined


# SCRIPT 8: Pipeline Class

In [13]:
class StructuralPreprocessingPipeline:
    def __init__(self, subject_id, input_file, output_dir='derivatives'):
        self.subject_id = subject_id
        self.input_file = input_file
        self.output_dir = Path(output_dir) / subject_id
        self.output_dir.mkdir(parents=True, exist_ok=True)
    
    def run(self):
        try:
            print(f"Processing {self.subject_id}...")
            orig = nib.load(self.input_file)
            reor = reorient_img(orig)
            corr = correct_bias(reor)
            brain, mask = strip_skull(corr)
            seg = segment_img(brain, mask)
            
            nib.save(brain, self.output_dir/f'{self.subject_id}_brain.nii.gz')
            nib.save(seg, self.output_dir/f'{self.subject_id}_seg.nii.gz')
            
            vv = np.prod(orig.header.get_zooms())
            sd = seg.get_fdata()
            vols = {t: float(np.sum(sd==l)*vv/1000) for t,l in [('CSF',1),('GM',2),('WM',3)]}
            vols['TIV'] = sum(vols.values())
            
            d = brain.get_fdata()
            md = mask.get_fdata().astype(bool)
            snr = d[md].mean() / d[~md].std() if d[~md].std()>0 else 0
            
            qc = {'subject_id': self.subject_id, 'snr': float(snr), 'volumes': vols, 'timestamp': datetime.now().isoformat()}
            with open(self.output_dir/f'{self.subject_id}_qc.json', 'w') as f:
                json.dump(qc, f, indent=2)
            
            print(f"✓ {self.subject_id} done")
            return {'status': 'success', 'output_dir': str(self.output_dir)}
        except Exception as e:
            print(f"✗ {self.subject_id} failed: {e}")
            return {'status': 'failed', 'error': str(e)}

print("Pipeline class defined")

Pipeline class defined


# SCRIPT 9: Batch Processing

In [14]:
def process_subject(info):
    p = StructuralPreprocessingPipeline(info['subject_id'], info['input_file'])
    return p.run()

def batch_process(subjects_csv, n_jobs=2):
    df = pd.read_csv(subjects_csv)
    with ProcessPoolExecutor(max_workers=n_jobs) as ex:
        results = list(ex.map(process_subject, df.to_dict('records')))
    rdf = pd.DataFrame(results)
    rdf.to_csv('results.csv', index=False)
    print(f"Success: {(rdf['status']=='success').sum()}/{len(rdf)}")
    return rdf

print("Batch functions defined")

Batch functions defined


# SCRIPT 10: Validation

In [16]:
def validate(subject_dir):
    sd = Path(subject_dir)
    rep = {'subject_id': sd.name, 'checks': {}, 'warnings': [], 'errors': []}
    for r in ['_brain.nii.gz', '_seg.nii.gz', '_qc.json']:
        if not list(sd.glob(f'*{r}')):
            rep['errors'].append(f"Missing {r}")
        else:
            rep['checks'][r] = 'OK'
    rep['status'] = 'FAILED' if rep['errors'] else 'PASSED'
    return rep

print("Validation defined")
print("\n=" * 10)
print("SCRIPTS 8-10 COMPLETE")
print("=" * 50)

Validation defined

=
=
=
=
=
=
=
=
=
=
SCRIPTS 8-10 COMPLETE
